In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import optimizers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, LSTM

/home/tomasz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



There are two types of models in keras. Sequential an Functional. Sequential is only one direction of computing (create layers in sequence), alternative is Functional. Functional models are defined by creating instances of layers and connecting them directly to each other in pairs, and then defining a Model that specifies the layers to act as the input and output to the model, via the parameters inputs and outputs, respectively.


In [ ]:
## print all layers
# [layer for layer in dir(layers) if not layer.startswith('_')]

In [ ]:
## print all optimizers
# [opt for opt in dir(optimizers) if not opt.startswith('_')]

## Sequential

First initialize the model with the sequential API. Example: https://www.tensorflow.org/guide/keras

In [2]:
model = Sequential(name='DummyModel')

we can immidiatelly add hidden layers using model.add()

In [12]:
help(model.add)

Help on method add in module tensorflow.python.keras.engine.sequential:

add(layer) method of tensorflow.python.keras.engine.sequential.Sequential instance
    Adds a layer instance on top of the layer stack.
    
    Arguments:
        layer: layer instance.
    
    Raises:
        TypeError: If `layer` is not a layer instance.
        ValueError: In case the `layer` argument does not
            know its input shape.
        ValueError: In case the `layer` argument has
            multiple output tensors, or is already connected
            somewhere else (forbidden in `Sequential` models).



In [ ]:
# help(layers.Dense)

create the firts hidden layer of 3 nodes

In [3]:
model.add(layers.Dense(name='FullyConnected_1', units=3, activation='relu', input_dim=2))  # dim == 2 -> two imputs nodes and 3 units (nodes)

you can skip input_dim (like below) then it will be determined from the data given to model. Safer is to give it explicitly (less error prone). In previous version it raises ValueError

In [9]:
# model.add(layers.Dense(name='FullyConnected_1', units=3, activation='relu'))

Parameters: 2*3 weights + 3 bias

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FullyConnected_1 (Dense)     (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


you can define layers explicitly (activation as a separate layer)

In [5]:
# output layer
model.add(layers.Dense(name='OutputLayer_2', units=1))
model.add(layers.Activation(name='Sigmoid_2', activation='sigmoid'))
model.add(Dropout(0.2))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FullyConnected_1 (Dense)     (None, 3)                 9         
_________________________________________________________________
OutputLayer_2 (Dense)        (None, 1)                 4         
_________________________________________________________________
Sigmoid_2 (Activation)       (None, 1)                 0         
_________________________________________________________________
dropout (Dropout)            (None, 1)                 0         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [10]:
# model:

#   * \
#   * - * - * (activation) - * (dropout)
#   * /

After crating model it must be compiled

In [6]:
model.compile(
    optimizer='Adam',
    loss='binary_crossentropy', # binary classification
    metrics=['accuracy']
)

# example of usage

# summary = model.fit(
#     X_train, y_train,
#     batch_size=100,
#     epochs=10,
#     validation_split=0.1,
#     verbose=1,  # silent mode: 0
#     callbacks=[func1, func2]  # functions called by callback during learning
# )

If you really want the complete picture, check the configuration.

In [7]:
model.get_config()

[{'class_name': 'Dense',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 2),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'dtype': 'float32',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'FullyConnected_1',
   'trainable': True,
   'units': 3,
   'use_bias': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distributi

## Functional
Alternative is functional API

In [15]:
# hidden_3_lstm = LSTM(units=3, dropout=0.1, return_sequences=True)(hidden_2_dense)

In [ ]:
# example from: https://www.tensorflow.org/api_docs/python/tf/keras/Model

# import tensorflow as tf

# inputs = tf.keras.Input(shape=(3,))
# x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
# outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs

In [2]:
input_1 = Input(name="Input_1", shape=(3,)) # the expected input will be batches of 3-dimensional vectors.

hidden_2_dense = Dense(name="FullyConnected_2", units=3)(input_1)
hidden_2_norm = layers.BatchNormalization(name="Normalization_2")(hidden_2_dense)  # normalization of each layer and feature independently 
hidden_2_activ = Activation(name='ReLU_2', activation='relu')(hidden_2_norm)

model2 = Model(inputs=input_1, outputs=hidden_2_activ)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_1 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
FullyConnected_2 (Dense)     (None, 3)                 12        
_________________________________________________________________
Normalization_2 (BatchNormal (None, 3)                 12        
_________________________________________________________________
ReLU_2 (Activation)          (None, 3)                 0         
Total params: 24
Trainable params: 18
Non-trainable params: 6
_________________________________________________________________


## Save model

maybe needed (Debian): sudo apt-get install libhdf5-serial-dev

In [4]:
model2.compile(
    optimizer='Adam',
    loss='binary_crossentropy', # binary classification
    metrics=['accuracy']
)

In [5]:
model2.save("my_model.h5") # save all

In [8]:
model3 = tf.keras.models.load_model("my_model.h5")
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_1 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
FullyConnected_2 (Dense)     (None, 3)                 12        
_________________________________________________________________
Normalization_2 (BatchNormal (None, 3)                 12        
_________________________________________________________________
ReLU_2 (Activation)          (None, 3)                 0         
Total params: 24
Trainable params: 18
Non-trainable params: 6
_________________________________________________________________


save only architecture, and not its weights or its training configuration

In [ ]:
# save as JSON
json_string = model.to_json()
# save as YAML
yaml_string = model.to_yaml()

# model reconstruction from JSON:
model = tf.keras.models.model_from_json(json_string)
# model reconstruction from YAML
model = tf.keras.models.model_from_yaml(yaml_string)

save weights of a model

In [ ]:
model.save_weights('my_model_weights.h5')
model.load_weights('my_model_weights.h5')

If you need to load weights into a different architecture (with some layers in common), for instance for fine-tuning or transfer-learning, you can load weights by layer name:

In [ ]:
model.load_weights('my_model_weights.h5', by_name=True)